In [3]:
import gradio as gr
import tiktoken
from joblib import load
import numpy as np

In [2]:
def do_something(name, age):
    return f'Hello, {name}', age

def do_anotherthing(country):
    return f'You are from {country}'
    
# Tab 1
app1 =  gr.Interface(fn = do_something,
                      title = 'Interface 1',
                      inputs = ['text','number'], 
                      outputs = ['text','number'])

# Tab 2
app2 =  gr.Interface(fn = do_anotherthing,
                      title = 'Interface 2',
                      inputs = 'text', 
                      outputs = 'text')

# grouping Tab 1 and 2
tabbed = gr.TabbedInterface([app1,app2],
                             ['Greeting','Country'])
tabbed.launch()

c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\blocks.py:959: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
# ================== #
# Tokenize Text
# ================== #

# TODO: change tiktoken out to real tokenizer... or use tiktoken

def get_tokens(text_2_encode: str, tokenizer=None):
    """
    Tokenize text in a string.

    Initialize a tokenizer if tokenizer == None.
    """

    if tokenizer is None:
        tokenizer = tiktoken.encoding_for_model("text-davinci-003")
    return tokenizer.encode(text=text_2_encode)


def get_num_tokens(text_2_encode: str, **kwargs):
    """
    Count the number of tokens in a string.
    """
    return len(get_tokens(text_2_encode=text_2_encode, **kwargs))


# ================== #
#  Get Embeddings
# ================== #

def get_embeddings(text=None, model=None):
    """
    Generate embeddings on a string of text.
    """
    if model==None:
        model = load('./model/SentBERTmodel.pkl')

    return model.encode(text)


# ================== #
#  Calculate Vector Similarity
# ================== #

def vector_similarity(x: "list[float]", y: "list[float]") -> float:
    """
    Returns the similarity between two vectors.

    Because embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


# ================== #
#  Order Chunks by Similarity
# ================== #

def measure_embedding_similarity(
    query: str,
    embeddings
):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embeddings(query)

    return [vector_similarity(query_embedding, embedding) for embedding in embeddings]


# ================== #
#  Get Similar Texts
# ================== #

def get_similar_texts(df, k):
    """
    Slice a dataframe on the top k results.  Sort the sliced dataframe descending on similarity score.

    If there are repeated results in top 5, keep them all.
    """
    response = df.nlargest(k, columns=['similarity score'],keep='all')
    response = response.sort_values(by='similarity score', ascending=False)
    return response

In [5]:
# ================== #
#  Run (score.py)
# ================== #

def run(question: str, k: int, embeddings, df) -> dict[str,str]:

    # Retrieve Top K Most Similar Results
    df['similarity score'] = measure_embedding_similarity(question, embeddings)

    # Count number of tokens in each article
    df['token count'] = df['text'].apply(get_num_tokens)
    
    # Return Chunks With Highest Similarity (Text)
    response = get_similar_texts(df, k)

    # Remove embeddings column
    keep_columns = ['book', "chapter", 'text', 'token count', 'similarity score']
    response = response[keep_columns]
    
    return response

In [6]:
# Creating a Gradio interface for the chatbot
with gr.Blocks() as demo:
    title = gr.HTML("<h1>Bible Reference Assistant</h1>")
    input = gr.Textbox(label="What would you like to know?")  # Textbox for user input
    output = gr.Textbox(label="Here ya go, Champ:")  # Textbox for chatbot response
    btn = gr.Button("Gimme the answer")  # Button to trigger the agent call
    btn.click(fn=run, inputs=input, outputs=output)

# Launching the Gradio interface
demo.launch(share=True, debug=True)

c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\utils.py:816: UserWarning: Expected 4 arguments for function <function run at 0x000002A4B3F7DCA0>, received 1.
  warnings.warn(
c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\utils.py:820: UserWarning: Expected at least 4 arguments for function <function run at 0x000002A4B3F7DCA0>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://64918e97c22f01a8d1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\helpers.py:796: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
Traceback (most recent call last):
  File "c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\queueing.py", line 427, in call_prediction
    output = await route_utils.call_process_api(
  File "c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\blocks.py", line 1497, in process_api
    result = await self.call_function(
  File "c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\gradio\blocks.py", line 1119, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return a